<a href="https://colab.research.google.com/github/milindkhare/milindkhare/blob/Pypractice/Web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Error Handling in Py

In [ ]:
## Error handling
for i in range(2):
  try:
    print("1")
    print(2)
    print(hey)
  #except:
  except Exception as e:
    print("Dude:", e)

print('yo man')

1
2
Dude: name 'hey' is not defined
1
2
Dude: name 'hey' is not defined
yo man


# Web scrapping in Python

Using Requests and Beautiful Soup for mining Apple prev close share price

In [ ]:
import requests

url = "https://finance.yahoo.com/quote/AAPL?p=AAPL"

response = requests.get(url)
print(response)

  text = response.text

  #This is obtained by inspect html of url
  prop = "Previous Close"

# Find index where prop begins
prop_ind = text.index(prop) 

#split till first span and then take out value of prev close
#this part is done after looking manually at isnpect html of the page
reduced_text = text[prop_ind:].split("</span>")[1]
prop_value = reduced_text.split(">")[-1]
print(prop_value)

<Response [200]>
148.12


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://finance.yahoo.com/quote/ALLE?p=ALLE"

response = requests.get(url)
text = response.text
#print(text)
soup = BeautifulSoup(text, features='html.parser')
trs = soup.find_all("tr") #This is obtained by inspect html of url
print(trs)
# print(trs[0].contents[0].text)
# print(trs[0].contents[1].text)
# print(trs[1].contents[0].text)
# print(trs[1].contents[1].text)
# print(range(len(trs)))

# Retrieve all content for 1 stock

#names = []
#values = []
# nameval = {} # Dictionary of key and value pair (Names can directly used as index for value)

for i in range(len(trs)):
  for j in range(len(trs[i].contents)):
    if j == 0: #first is name
      try:
        name = trs[i].contents[j].text
        #names.append(name)
      except:
        continue #move on to next block
    if j == 1: #second is value
      try:
        value = trs[i].contents[j].text
        #values.append(value)
      except:
        continue #move on to next block
  #add it in dict
  nameval[name] = value
  if name == '1y Target Est': #final name in the table
    break

# print(nameval)
#print(names) print(values)
#names() and values can be used to store vector data seperately if needed

Fetch all S&P500 stocks data, make it repeated run

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time,os,datetime

wikiurl = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = requests.get(wikiurl)
pagetext = response.text

# First we try to find all the ticker symbols from wiki page -----
wikisoup = BeautifulSoup(pagetext, features='html.parser')
tbody = wikisoup.find_all("tbody") #This is obtained by inspect html of url

#print(tbody[0].contents[2])
# every even content has ticker symbol

## Retrieve all tickers
tickersymbols = []
endsymbol = 'ZTS'

for i in range(len(tbody[0].contents)):
  if i<2:
    continue
  if i%2 != 0:
  #if int(i/2)!=i/2:
    continue
  # fetch symbol
  symbol = tbody[0].contents[i].contents[1].text
  tickersymbols.append(symbol.strip("\n"))

  if symbol.strip("\n") == endsymbol: #final name in the table
    break

print(tickersymbols)

# Define function for fetching yahoo fin data -----
def getFinancialInformation(symbol):
  url = "https://finance.yahoo.com/quote/"+symbol+"?p="+symbol+""

  response = requests.get(url)
  text = response.text
  
  soup = BeautifulSoup(text, features='html.parser')
  trs = soup.find_all("tr") #This is obtained by inspect html of url
  
  # Retrieve all content for stock
  names = []
  values = []
  #nameval = {} # Dictionary of key and value pair (Names can directly used as index for value)

  for i in range(len(trs)):
    for j in range(len(trs[i].contents)):
      if j == 0: #first is name
        try:
          name = trs[i].contents[j].text
          names.append(name)
        except:
          continue #move on to next block
      if j == 1: #second is value
        try:
          value = trs[i].contents[j].text
          values.append(value)
        except:
          continue #move on to next block
      
      #print(name)
      #print("/n")
    
  #add it in dict
  #nameval[name] = value
    if name == '1y Target Est': #final name in the table
     break
  
  #print(names)
  return names,values

# Now use ticker symbols to fetch all yahoo fin data -----

# and make it run every 15 min
# append data
# store time stamp of values

while True:
  # check current data
  start = time.time()
  reruntime = 900 #15 min

  # Extract and save data
  data = {"symbol":[],
          "metric":[],
          "value":[],
          "time"=[]}

  for symbol in tickersymbols:
    print(symbol)
    try:
      names,values = getFinancialInformation(symbol) #retrieve data for symbol
      fetchtime = str(datetime.datetime.now())
    except Exception as e:
      print("fetch failed for : "+symbol)
      print("error: "+str(e))
      continue

    #store each names value as tuple under symbol
    for i in range(len(names)):
      try:      
        data["symbol"].append(symbol)
        data["metric"].append(names[i])
        data["value"].append(values[i])
        data["time"].append(fetchtime)
        #print(data)
      except:
        print(i)

    # another way to create data without for loop
    #data["symbol"] +=.append(symbol) 
    #data["metric"] += names
    #data["value"] += values

  #Create dataframe
  df = pd.DataFrame(data)
  date = datetime.date.today()
  savepath = str(date)+"S&P data.csv"
  if od.path.isfile(savepath):
    #dont overwrite, append
    df.to_csv(savepath, mode='a', header=False, columns=["symbol", "metric", "value"])
  else:
    #create
    df.to_csv(savepath, columns=["symbol", "metric", "value"])

  # wait until rerun time
  timediff = time.time() - start
  if timediff > reruntime: #900 sec
    time.sleep(reruntime)



In [ ]:
#tickersymbols[0:2]
#getFinancialInformation("ALLE")
df = pd.read_csv("S&P data.csv", index_col =0)
df.head()
df.iloc[-1]

symbol              ZTS
metric    1y Target Est
value            225.00
Name: 7151, dtype: object

Web scrapping Exercise-
  url = "http://webscraper.io/test-sites/tables" has tables. Extract all #,Fname, Lname and username and put in pd df.

In [ ]:
import requests
import pandas as pd
url = "http://webscraper.io/test-sites/tables"

response = requests.get(url)
#print(response.text)#Always visualizing our progress
#Each value in the table is surrounded by <tr> </tr>
tables = response.text.split("<tr>")
counter = 0

for table in tables:
    if "Mark" in table:#Seeing where data tarts
        print("Mark:",counter)
    if "@twitter" in table:#Seeing where data ends
        print("Twitter:",counter)
    counter+=1
    
tables = tables[2:19] #Reducing our data down to what's interesting
#print(tables)#Always visualizing our progress
reducedTables = []
for table in tables:
    if "<td>" in table:
        reducedTables.append(table)
#print(reducedTables)#Always visualizing our progress

doubleReducedTables = []
for table in reducedTables:
    temp = table.split("<td>")
    for tableTemp in temp:
        if "</td>" in tableTemp:
            #Here we make the first part of the string be the #, name, last name, username
            doubleReducedTables.append(tableTemp) 
#print(doubleReducedTables)#Always visualizing our progress
#To store out data in a neat fashion
data = {"#":[],
        "First Name":[],
        "Last Name":[],
        "Username":[]}
for i in range(len(doubleReducedTables)):
    table = doubleReducedTables[i]
    t = i%4 #Cycling over 4 different values
    value = table.split("</td>")[0]
    #Our data is always before the </td> value, so the first element after the split
    if value != "-":#We can take out the empty line
        if t == 0:
            data["#"].append(value)
        elif t == 1:
            data["First Name"].append(value)
        elif t == 2:
            data["Last Name"].append(value)
        elif t == 3:
            data["Username"].append(value)
    
df = pd.DataFrame(data)
print(df)

Mark: 2
Twitter: 4
Mark: 10
Twitter: 12
Mark: 16
Twitter: 18
    # First Name Last Name  Username
0   1       Mark      Otto      @mdo
1   2      Jacob  Thornton      @fat
2   3      Larry  the Bird  @twitter
3   4      Harry    Potter       @hp
4   5       John      Snow    @dunno
5   6        Tim      Bean  @timbean
6   1       Mark      Otto      @mdo
7   2      Jacob  Thornton      @fat
8   3      Larry  the Bird  @twitter
9   1       Mark      Otto      @mdo
10  2      Jacob  Thornton      @fat
11  3      Larry  the Bird  @twitter


# Scrapping AJAX content

In [ ]:
#url = "http://testing-ground.scraping.pro/ajax"
#from selenium import webdriver
#Try your code here
#You can see the solution in the Solution.txt file on the left side

url = "http://testing-ground.scraping.pro/ajax"
from selenium import webdriver

def findXPath(element,target,path):
    if target in element.get_attribute('textContent') and element.tag_name == "ul":
        return path
    newElements = element.find_elements_by_xpath("./*")
    for newElement in newElements:
#        print(path+ "/"+newElement.tag_name)
        final = findXPath(newElement,target,path + "/"+newElement.tag_name)
        if final != "":
            return final
    return ""
browser = webdriver.PhantomJS(executable_path='/usr/local/bin/phantomjs')
browser.get(url)
#print(browser.page_source)
elements = browser.find_element_by_xpath("html")
finalXPath = findXPath(elements,"Andrew","html")
print("Final xPath:",finalXPath)
element = browser.find_element_by_xpath(finalXPath)
print("Names:\n",element.text) #Fortunately here, we don't need to use get_attribute("textContent")
                                 #But you can try it out anyway to see the difference in formatting
browser.quit()